In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

from implementation import *

# Load and clean the data

In [2]:
y,x,ids=load_csv_data("train.csv") 

In [3]:
# tej colonnes où y'a des -999

x[x==-999]=np.NaN
z=x[:,~np.isnan(x).any(axis=0)] #z is our matrix of observations with size (number of observations, number of features without -999)
#z = build_poly(z,5)

In [4]:
# cross validation for 2 --> TO DO: FOR K

Strain1_z, Stest1_z, Strain1_y, Stest1_y = split_data(z,y,0.8)

# standardize the data (only centered here)

Strain1_z = standardize(Strain1_z)
Stest1_z = standardize(Stest1_z)
Strain1_y = standardize(Strain1_y)
Stest_y = standardize(Stest1_y)

# Calculate MSE for result from least squares 

In [6]:
beta1 = least_squares(Strain1_y, Strain1_z)      #beta from y = X^T*beta --> missing the column of ones'
print(beta1)
res1 = definitive_res(Stest1_z.dot(beta1))
print(res1)
len(Stest1_y[Stest1_y==res1])/len(Stest1_y)

[ -2.72467702e-01  -2.73909021e-01   7.02354261e-03   2.99904871e-01
  -4.31654279e-02  -7.52264692e+02  -2.02161073e-01   1.41353721e-01
   1.45888169e+02  -5.48414141e-04  -1.19720570e-03   1.43839291e+02
  -4.88301384e-04   2.83424132e-03   1.10759427e-01   2.08260539e-04
  -7.64132051e-02   8.76114707e-02   6.37387796e+02]
[-1. -1. -1. ...,  1.  1. -1.]


0.64518

# Calculate MSE for results from least squares GD and SGD

In [7]:
initial_w = np.zeros(Strain1_z.shape[1])    #w is an array of size = number of features (i.e. number of columns in z)
max_iters = 50
gamma = 0.7

# GD

In [8]:
beta2 = least_squares_GD(Strain1_y, Strain1_z, initial_w, max_iters, gamma)      #beta from y = X*beta --> missing the column of ones'
res2 = definitive_res(Stest1_z.dot(beta2))
len(Stest1_y[Stest1_y==res2])/len(Stest1_y)

0.37736

# SGD

In [9]:
batch_size = 1

beta3 = least_squares_SGD(Strain1_y, Strain1_z, initial_w, batch_size, max_iters, gamma)      #beta from y = X*beta --> missing the column of ones'
res3 = definitive_res(Stest1_z.dot(beta3))
len(Stest1_y[Stest1_y==res3])/len(Stest1_y)

0.47282

# Calculate other LS for Ridge Regression 

In [10]:
lambdas = np.logspace(-5, 0, 15)

# bluid poly
tx_tr = build_poly(Strain1_z, 1)
tx_te = build_poly(Stest1_z, 1)

# ridge regression with different lambda
rmse_tr = []
rmse_te = []
for ind, lambda_ in enumerate(lambdas):
    # ridge regression
    weight = ridge_regression(Strain1_y, tx_tr, lambda_)
    rmse_tr.append(np.sqrt(2 * compute_mse(Strain1_y, tx_tr, weight)))
    rmse_te.append(np.sqrt(2 * compute_mse(Stest1_y, tx_te, weight)))
print(rmse_tr, rmse_te)
print("last weight:",weight)

[0.88460840887094994, 0.88460841183365735, 0.88460842713737819, 0.88460850618385956, 0.88460891315894163, 0.88461099273034471, 0.88462144043193258, 0.88467200926605549, 0.88489824925926908, 0.88576676290789547, 0.88834725805310177, 0.89383560367066051, 0.90261781867698754, 0.91536939356743507, 0.93321807541987101] [0.89664252680744594, 0.89664218560990216, 0.89664141792818453, 0.89663971487044136, 0.89663606464128953, 0.89662890361463332, 0.89661829110114, 0.8966208439332447, 0.89673802049874907, 0.89736426657322677, 0.89944342939083666, 0.90402639283308306, 0.91141360460448206, 0.92229805340802329, 0.9380305617806427]
last weight: [  2.20810229e-15  -9.37192183e-02  -5.58337460e-03   3.31217333e-02
   2.72850659e-02  -1.78938890e-02   1.83299369e-02  -4.67054298e-02
   6.41311460e-02   5.56090925e-02  -4.43533135e-04  -1.40247489e-03
  -1.00167291e-03   3.77606524e-05   1.29673439e-03  -3.82863638e-03
   1.27019992e-03   1.00192766e-02   1.38183170e-02   9.14801319e-03]
